# Import Packages

In [ ]:
import os, torch
import numpy as np
import pandas as pd
from PIL import Image
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from utils import *

# Setup

In [ ]:
csv_file = "train.csv"
data_list = pd.read_csv(csv_file)
data_root = "train"
file = "{}.jpg"
data_list

# 0. Pipeline Illustration
關於訓練模型，我們有幾個步驟要做：
1. 讀資料、建構 Dataset 及 DataLoader
2. 定義 model, optimizer, loss
3. training and validation

# 1. Load Data and Construct Dataset, DataLoader

### 1.1. Load JPG, JPEG, PNG
因為我們的影像是 JPEG 檔，所以我們來學如何讀這系列的檔案

##### Note.
這系列の檔案類型都是色彩強度在 [0, 255] 的影像

In [ ]:
data_list.StudyInstanceUID

In [ ]:
filename = "1.2.826.0.1.3680043.8.498.16451034714945708059993280774682419855.jpg"
filename

In [ ]:
data_root + "/" + filename

In [ ]:
root = "train/"
root + "/" + filename

In [ ]:
os.path.join(root, filename)

In [ ]:
os.path.join(data_root, filename)

In [ ]:
example_image = np.random.choice(data_list.StudyInstanceUID)

os.path.join(data_root, example_image)

In [ ]:
example_image = np.random.choice(data_list.StudyInstanceUID)
path = os.path.join(data_root, file.format(example_image))
path

In [ ]:
image = Image.open(path)
image

In [ ]:
array = np.array(image)
print(f"shape = {array.shape}")
np.unique(array)

In [ ]:
import nibabel as nib
# define path
nifti = nib.load(path)
array = nifti.get_fdata()

### 1.2. Pytorch Dataset and DataLoader
Pytorch Dataset 是一種 iterable（熟悉吧？）
1. 它必須繼承 Pytorch 的 Dataset class
2. 是一個 iterable class，會一個一個吐出你的 data

至於 data 要用什麼形式包裝他就不限制了。
不過我想給大家一個管理 data 的建議：一個 data 用一個 dictionary 包裝。

In [ ]:
# data_list.iloc[0, 0]
# data_list.loc[0, "StudyInstanceUID"]

In [ ]:
example_index = 0
data_list.iloc[example_index, 1:-1]

In [ ]:
data_list.iloc[example_index, 1:-1].values

In [ ]:
np.array(data_list.iloc[example_index, 1:-1])

In [ ]:
np.array(data_list.iloc[example_index, 1:-1], dtype="float32")

In [ ]:
example_index = 0
data = {
    "patient_id": data_list.iloc[example_index, -1],
    "image": os.path.join(data_root, file.format(data_list.iloc[example_index, 0])),
    "label": np.array(data_list.iloc[example_index, 1:-1], dtype="float32")
}
data

### 1.3. Your Trun!
請你動手寫一個名為 RANZCR 的 Pytorch Dataset。
1. 繼承 torch.utils.data.Dataset (Hint: 在開頭的 import 環節，我已經幫你把它 import 成 Dataset 了，繼承 Dataset 並 initialize 就好）
2. initialize 兩個字串 data_root 和 csv_file（建議按照順序）
  * 把 csv_file 讀成 pandas DataFrame 作為 attribute
  * 直接把 data_root 存成字串，用來讀串接影像名稱
3. 用你的方式寫一個 iterable：
  * 放到迴圈時，每一次迭代吐一組資料（即一個 dictionary 如上）
  * 可以取長度
  * 可以多次放到迴圈裡面使用

In [ ]:
import tensorflow as tf
tf. ...
import tensorflow
tensorflow. ...

In [ ]:
# your code here
class RANZCR(Dataset):
    def __init__(self, data_root, csv_file):
        super(RANZCR, self).__init__()
        self.data_root = data_root
        self.data_list = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, i):
        data = {
            "patient_id": self.data_list.iloc[i, -1],
            "image": os.path.join(self.data_root, file.format(self.data_list.iloc[i, 0])),
            "label": np.array(self.data_list.iloc[i, 1:-1], dtype="float32")
        }

        return data

In [ ]:
check_answer = True

if check_answer:
    dataset = RANZCR(data_root, csv_file)

    for data in dataset:
        print(data)
        break

### 1.4. Data Transform
你肯定注意到了，我們讀進來的資料只有檔名，不可能直接拿來 train。
這時我們需要一連串的 transforms 來讓資料從檔名開始經歷他的奇幻旅程，這中間你愛加多少 data augmentation 都隨你開心。

In [ ]:
# 帶大家寫 Tranform 以及把 Dataset 修成有 transform 的版本

class RANZCR(Dataset):
    def __init__(self, data_root, csv_file, transform=None):
        super(RANZCR, self).__init__()
        self.data_root = data_root
        self.data_list = pd.read_csv(csv_file)
        self.transform = transform
        self.file = "{}.jpg"

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, i):
        data = {
            "patient_id": self.data_list.iloc[i, -1],
            "image": os.path.join(self.data_root, file.format(self.data_list.iloc[i, 0])),
            "label": np.array(self.data_list.iloc[i, 1:-1], dtype="float32")
        }

        if self.transform is not None:
            data = self.transform(data)

        return data

In [ ]:
class JPGLoader(Transform):
    def __init__(self, keys):
        self.keys = keys

    def __call__(self, data):
        for key in self.keys:
            if key in data:
                path = data[key]
                data[key] = Image.open(path)

            else:
                raise KeyError(f"{key} is a key of {data}.")

        return data

In [ ]:
raise IndexError()

In [ ]:
raise KeyError()

In [ ]:
raise StopIteration()

In [ ]:
raise RuntimeError()

In [ ]:
dataset = RANZCR(data_root, csv_file, transform=JPGLoader(keys=["imae"]))

for data in dataset:
    print(data)
    break

In [ ]:
dataset = RANZCR(data_root, csv_file, transform=JPGLoader(keys=["image"]))

for data in dataset:
    print(data)
    break

In [ ]:
data["image"].shape

In [ ]:
image = data["image"]
resize_transform = transforms.Resize((224, 224), interpolation=2)
resize_transform(image)

In [ ]:
class Resize(Transform):
    def __init__(self, keys, size, interpolation=2):
        self.keys = keys
        self.resize = transforms.Resize(size, interpolation=interpolation)

    def __call__(self, data):
        for key in self.keys:
            if key in data:
                image = data[key]
                data[key] = self.resize(image)

            else:
                raise KeyError(f"{key} is a key of {data}.")

        return data

class PILToTensor(Transform):
    def __init__(self, keys):
        self.keys = keys
        self.to_tensor = transforms.ToTensor()

    def __call__(self, data):
        for key in self.keys:
            if key in data:
                data[key] = self.to_tensor(data[key])

            else:
                raise KeyError(f"{key} is a key of {data}.")

        return data

class NumpyToTensor(Transform):
    def __init__(self, keys):
        self.keys = keys

    def __call__(self, data):
        for key in self.keys:
            if key in data:
                data[key] = torch.Tensor(data[key])

            else:
                raise KeyError(f"{key} is a key of {data}.")

        return data

In [ ]:
transform = transforms.Compose([
    JPGLoader(keys=["image"]),
    Resize(keys=["image"], size=(224, 224), interpolation=2),
    PILToTensor(keys=["image"]),
    NumpyToTensor(keys=["label"])
])
dataset = RANZCR(data_root, csv_file, transform=transform)

data = dataset[0]
print(data)
# for data in dataset[:2]:
#     print(data)
#     break

{'patient_id': 'ec89415d1', 'image': tensor([[[0.0157, 0.0157, 0.0157,  ..., 0.0392, 0.0392, 0.0353],
         [0.0588, 0.0588, 0.0549,  ..., 0.2000, 0.1922, 0.1725],
         [0.0510, 0.0549, 0.0510,  ..., 0.2471, 0.2353, 0.2157],
         ...,
         [0.2353, 0.2824, 0.2706,  ..., 0.5216, 0.4627, 0.3843],
         [0.2392, 0.2784, 0.2510,  ..., 0.5294, 0.4706, 0.3961],
         [0.2588, 0.2667, 0.2235,  ..., 0.4980, 0.4431, 0.3725]]]), 'label': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])}


In [ ]:
data["label"].shape

torch.Size([11])

### 1.5. DataLoader
DataLoader 是 Pytorch 一個很方便的物件，它直接幫你把 Dataset 變成可以 batch-wise 讀取的迭代器，同時實現平行化讀取。

完整的 shape 應該是 $\text{batch} \times \text{channels} \times \text{height} \times \text{width}$

In [ ]:
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

In [ ]:
# data = loader[0]
# print(data["image"].shape)

In [ ]:
for data in loader:
    print(data["image"].shape)
    break

torch.Size([32, 1, 224, 224])


# 2. Setup Hyperparameters

### 2.1. Model Construction
Pytorhc model 是一種 torch.nn.Module class，並且有定義 __init__, forward
* __init__ 用來存 parameters
* forward 吃一個 input x，你必須指明 x 會經過哪些運算，最後 output 出去

##### Note.
但因為我幫你把 nn import 好了，所以你只需要寫 nn.Module 就可以使用它了

In [ ]:
class ExampleModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ExampleModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=4, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=4, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=4, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.flat = nn.Flatten()
        self.linear = nn.Linear(256, out_channels)
        self.bn_out = nn.BatchNorm1d(out_channels)

    def call(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.flat(x)
        x = F.sigmoid(self.bn_out(self.linear(x)))

        return x

In [ ]:
model = ExampleModel(1, 11)
model

In [ ]:
x = torch.rand(32, 1, 128, 128)
y = model(x)
y.size()

In [ ]:
class ExampleModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ExampleModel, self).__init__()
        # input 128 x 128
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=4, padding=1) # 32 x 32
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=4, padding=1) # 8 x 8
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=4, padding=1) # 2 x 2
        self.bn3 = nn.BatchNorm2d(64)
        self.flat = nn.Flatten() # 64 x 2 x 2 = 256
        self.linear = nn.Linear(256, out_channels)
        self.bn_out = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.flat(x)
        x = torch.sigmoid(self.bn_out(self.linear(x)))

        return x

In [ ]:
model = ExampleModel(1, 11)
model

In [ ]:
x = torch.rand(32, 1, 128, 128)
y = model(x)
y.size()

In [ ]:
class ExampleModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ExampleModel, self).__init__()
        # input 224 x 224
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=4, padding=1) # 56 x 56
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=4, padding=1) # 14 x 14
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0) # 12 x 12
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3, stride=4, padding=1) # 3 x 3
        self.bn4 = nn.BatchNorm2d(64)
        self.flat = nn.Flatten() # 64 x 3 x 3 = 576
        self.linear = nn.Linear(576, out_channels)
        self.bn_out = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.flat(x)
        x = torch.sigmoid(self.bn_out(self.linear(x)))

        return x

In [ ]:
model = ExampleModel(1, 11)
model

In [ ]:
x = torch.rand(32, 1, 224, 224)
y = model(x)
y.size()

### 2.2. Optimizers and Losses

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

### 2.3. GPU Acceleration
1. 指定 device
  * device = torch.cuda.device("cuda:0" if torch.cuda.is_availabel() else "cpu")
2. 將 model, criterion, optimizer 放到 device
  * model.to(device)
  * criterion.to(device)
  * optimizer.to(device)

In [ ]:
"cuda:0" if torch.cuda.is_available() else "cpu:0"

'cpu:0'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu:0")
model = model.to(device)

In [ ]:
next(model.parameters()).is_cuda

False

# 3. Training and Validation

### 3.1. Training Process
當我們的 data loader, model, criterion, optimizer 都設置好以後，接下來的 training process 就可以用底下幾個步驟概括了

0. 清空 optimizer 的 gradient 及將 model 改為訓練模式
  * optimizer.zero_grad()
  * model.train()
1. 從 data loader 取得一組資料
  * 資料 input
  * 標註 target
2. 計算 model 的預測
  * output = model(input)
3. 計算 loss
  * loss = criterion(output, target)
4. 計算 gradient
  * loss.backward()
5. 更新參數
  * optimizer.step()

### 3.2. Your Turn!
這邊留給你們寫應該不過分吧XD

In [ ]:
label.shape

torch.Size([32, 1, 1, 11])

In [ ]:
a = torch.Tensor([[1, 1], [1, 0]])
b = torch.Tensor([[1, 1], [1, 1]])
a

tensor([[1., 1.],
        [1., 0.]])

In [ ]:
b

tensor([[1., 1.],
        [1., 1.]])

In [ ]:
a == b

tensor([[ True,  True],
        [ True, False]])

In [ ]:
n_correct = torch.sum(a == b, axis=0).double()
n_correct

tensor([2., 1.], dtype=torch.float64)

In [ ]:
mean_correct = torch.mean(n_correct) / len(a)

tensor(0.7500, dtype=torch.float64)

In [ ]:
len(torch.zeros([3, 2])), len(torch.zeros([2, 3]))

(3, 2)

In [ ]:
torch.mean(torch.sum((prob > 0.5) == label, axis=0).double()) / len(prob)

tensor(0.5994, dtype=torch.float64)

In [ ]:
len(prob), len(label)

(32, 32)

In [ ]:
def compute_accuracy(pred, label):
    n_correct = pred == label
    total_correct = torch.sum(n_correct, axis=0).double()
    mean_correct = torch.mean(total_correct)

    return mean_correct / len(pred)

In [ ]:
# your code here
from sklearn.metrics import roc_auc_score
# 0.1.
model.train()
n_steps = len(loader)

for i, data in enumerate(loader):
    # 0.2.
    optimizer.zero_grad()
    
    # 1.
    image = data["image"]
    label = data["label"]

    # 2.
    prob = model(image)

    # 3.
    loss = criterion(prob, label)

    # 4.
    loss.backward()

    # 5.
    optimizer.step()

    # bonus
    acc = compute_accuracy(prob > 0.5, label)

    print(f"step [{i}, {n_steps}], loss = {loss.detach().numpy()}, acc = {acc.detach().numpy()}")

step [0, 941], loss = 0.4702613055706024, acc = 0.8920454545454546
step [1, 941], loss = 0.49016091227531433, acc = 0.8806818181818182
step [2, 941], loss = 0.5011453628540039, acc = 0.8409090909090909
step [3, 941], loss = 0.4752653241157532, acc = 0.8721590909090909
step [4, 941], loss = 0.48357945680618286, acc = 0.8636363636363636
step [5, 941], loss = 0.48490336537361145, acc = 0.8522727272727273
step [6, 941], loss = 0.493107408285141, acc = 0.8522727272727273
step [7, 941], loss = 0.4924885034561157, acc = 0.8409090909090909
step [8, 941], loss = 0.5002603530883789, acc = 0.8352272727272727
step [9, 941], loss = 0.4823310971260071, acc = 0.8721590909090909


KeyboardInterrupt: 

### 3.3. Validation Process
同上設置，我們有底下步驟

0. 將 model 改為計算模式並且用 torch.no_grad() 包住整段 code
  * model.eval()
  * with torch.no_grad():
        ...
1. 從 data loader 取得一組資料
  * 資料 input
  * 標註 target
2. 計算 model 的預測
  * output = model(input)
3. 計算 loss 或你要的指標
  * loss = criterion(output, target)
  * metric = ...
4. 看你要 print 出來還是存到哪裡去都行

### 3.4. Your Turn!
就算你覺得過分我也不會理你 <3

In [ ]:
# your code here

model.eval()

with torch.no_grad():
    for data in loader:
        image = data["image"]
        label = data["label"]

        prob = model(image)
        loss = criterion(prob, label)

        print(f"loss = {loss.detach().numpy()}")

loss = 0.5063570141792297
loss = 0.5237624049186707
loss = 0.4740864038467407
loss = 0.5229761600494385
loss = 0.4966447651386261
loss = 0.5032168030738831
loss = 0.4962211847305298
loss = 0.49787452816963196
loss = 0.48986756801605225
loss = 0.4732462167739868
loss = 0.4749063551425934
loss = 0.48760679364204407
loss = 0.5086483955383301
loss = 0.5146855711936951
loss = 0.5227349400520325
loss = 0.5026934742927551
loss = 0.470589816570282
loss = 0.4690669775009155
loss = 0.48569098114967346
loss = 0.48486536741256714
loss = 0.5157176852226257
loss = 0.5140590071678162
loss = 0.4661915600299835
loss = 0.5472235679626465
loss = 0.5293269753456116
loss = 0.4919591248035431
loss = 0.5128269791603088
loss = 0.484343022108078
loss = 0.4893135130405426
loss = 0.5083281397819519
loss = 0.47075459361076355
loss = 0.5202732682228088
loss = 0.48119696974754333
loss = 0.4865117371082306
loss = 0.4798578917980194
loss = 0.48326605558395386
loss = 0.5060675144195557
loss = 0.474618136882782
loss = 

KeyboardInterrupt: 